## Assignment1-ML-for-NLP2
For this assignment, you will work in groups. (See group assignment below.) 
Use the samples of Twitter data posted in the Input folder (corresponding to NYT and Fox News) and do the following:


1. Data cleaning. 

Remove stopwords, links, hashtags, etc. 

2. Feature engineering.

Create popularity and engagement measures, mentions (i.e. a tweet includes @), popular hashtags, breaking news, growth of followers, an indicator if the tweet is posted by NYT, and zero if the tweet belongs to Fox News.

- popularity measures: likes, likes per retweets, likes per followers
- engagement measures: replies+quote_count, replies+quote_count per followers
- number of other accounts' mentions (@)
- popular hashtags
- tweets' special categories: breaking news, opinion
- is_nyt

3. Data Exploration and Prediction
- Is there a follower-tweet relationship? 
- What are the most frequent topics tweeted about by each outlet? Are there differences in outlets?
- Is there a relationship between the sentiment of the tweet and the outlet?
- Can we predict the outlet by the text of the tweet? Use classif micationethods (Naive Bayes, Logistic Regression, Support Vector Classifier using Count Vectorizer, Tf-Idf, or Word Embeddings). Which method performs better? Why?
- Do the usual word analogies hold in this sample?
- Create the most representative tweet by outlet (See  https://www.youtube.com/watch?v=zFScws0mb7M&t=23m03s and the Q&A for his description of it is implemented in the algorithm.)

Bonus 10 points: Create at least two own explorations.

Create a GitHub repository with your code and invite me to it. 


In [45]:
import numpy as np
from numpy import array
import pandas as pd
import regex as re

In [27]:
df1 = pd.read_csv('Inputs/FoxNews_all_tweets.csv')
df2 = pd.read_csv('Inputs/nytimes_all_tweets.csv')

# merging two datasets for simplicity
frames = [df1, df2]
df = pd.concat(frames, ignore_index= True)
df.head()

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,tweet_id,reference_status
0,1367531,FoxNews,20946694,456439,NaN,NaN,FLASHBACK: Ex-intel officials claimed the Hunt...,2022-03-18 15:50:00+00:00,26,39,68,0,1504847596006821894,NaN
1,1367531,FoxNews,20946694,456439,NaN,NaN,Biden must do more to counter Putin's propagan...,2022-03-18 15:45:00+00:00,23,94,92,7,1504846338097221635,NaN
2,1367531,FoxNews,20946694,456439,NaN,NaN,NASCAR: What to expect from Atlanta's super ne...,2022-03-18 15:40:00+00:00,10,9,21,0,1504845079344820224,NaN
3,1367531,FoxNews,20946694,456439,NaN,NaN,"BREAKING: Biden, Xi discuss Russia's war again...",2022-03-18 15:34:43+00:00,33,71,87,3,1504843752426323990,NaN
4,1367531,FoxNews,20946694,456439,NaN,NaN,Transgender swimmer's NCAA championship win 'j...,2022-03-18 15:30:00+00:00,51,69,254,4,1504842564117024770,NaN


In [28]:
df = df.drop(columns=['author_id', 'author_tweets', "author_description", "author_location", "tweet_id", "reference_status"])
df

,username,author_followers,text,created_at,retweets,replies,likes,quote_count
0,FoxNews,20946694,FLASHBACK: Ex-intel officials claimed the Hunt...,2022-03-18 15:50:00+00:00,26,39,68,0
1,FoxNews,20946694,Biden must do more to counter Putin's propagan...,2022-03-18 15:45:00+00:00,23,94,92,7
2,FoxNews,20946694,NASCAR: What to expect from Atlanta's super ne...,2022-03-18 15:40:00+00:00,10,9,21,0
3,FoxNews,20946694,"BREAKING: Biden, Xi discuss Russia's war again...",2022-03-18 15:34:43+00:00,33,71,87,3
4,FoxNews,20946694,Transgender swimmer's NCAA championship win 'j...,2022-03-18 15:30:00+00:00,51,69,254,4
...,...,...,...,...,...,...,...,...
1947,nytimes,52297191,"Kyrylo Lysov, 22, and his girlfriend Yelyzavet...",2022-03-11 16:40:18+00:00,737,170,5277,54
1948,nytimes,52297191,Our journalists are looking into how companies...,2022-03-11 16:30:12+00:00,32,38,150,7
1949,nytimes,52297191,Breaking News: Alec Baldwin gave his most deta...,2022-03-11 16:18:19+00:00,59,81,227,20
1950,nytimes,52297191,A shipwreck from 1891 has been discovered in L...,2022-03-11 16:10:04+00:00,88,15,361,16


## Feature Engineering

- Popularity and Engagement Measures:

We already have such popularity measures such as likes and retweets. We can create a few more popularity measures such as: likes per retweets, likes per followers (in millions). For engagement measures, we create variables such as sum of replies and quotes as well as the sum of replies and quotes per followers (in millions) to take into account different number of followers among two sources. 

In [24]:
df['likes_per_rt'] = df['likes']/df['retweets']
df['likes_per_mfllrs'] = df['likes']/df['author_followers']*1000000

df['repl_quot'] = df['replies']+df['quote_count']
df['repl_quot_per_mfllrs'] = df['repl_quot']/df['author_followers']*1000000

- Number of other accounts' mentions (@)

In [61]:
pattern =r'@([A-Za-z0-9_]+)'

df['mentions_count'] = df['text'].apply(lambda x: len(re.findall(pattern, x)))

- Two most popular tweets' special categories: breaking news, opinion

In [92]:
pattern1 = r'(BREAKING:)|(Breaking News:)'
df['is_breaking'] = df['text'].apply(lambda x: 1 if len(re.findall(pattern1, x))>0 else 0)

pattern2 = r'(OPINION:)|(In Opinion)'
df['is_opinion'] = df['text'].apply(lambda x: 1 if len(re.findall(pattern2, x))>0 else 0)

- Popular hashtags

Since there are only three distinctive hashtags among all tweets, we create dummy variables for each of them and also the dummy variable for the presence of any hashtag in text. 

In [99]:
pattern3 =r'\B(\#[a-zA-Z]+\b)(?!;)'
df['is_hashtag'] = df['text'].apply(lambda x: len(re.findall(pattern3, x)))

In [120]:
def return_all_hashtags(tweets):
    all_hashtags=[]
    for tweet in tweets:
        for word in tweet.split():
            if word.startswith('#'):
                word = re.sub(r'[^A-Za-z0-9 ]+', '', word) #remove all non‐alphanumeric characters except white space
                all_hashtags.append(word)      
    return all_hashtags

In [123]:
hashtag_list = return_all_hashtags(df['text'])
hashtag_list

['MeToo',
 'CriticsChoiceAwards',
 'CriticsChoiceAwards',
 'CriticsChoiceAwards',
 'CriticsChoice',
 'CriticsChoice',
 'CriticsChoice',
 'CriticsChoice',
 'CriticsChoice']

In [124]:
hashtag_set = set(hashtag_list)
hashtag_list = list(hashtag_set)
hashtag_list

['MeToo', 'CriticsChoice', 'CriticsChoiceAwards']

In [127]:
for hashtag in hashtag_list:
    #df2['hashtag_'+element] = np.where(('#'+element) in df2.text, 1,0)
    df['hashtag_'+hashtag] = df.text.apply(lambda x: 1 if hashtag in x else 0)

In [ ]:
df['is_nyt']= df['username'].apply(lambda x: len(re.findall(pattern3, x)))


In [128]:
df

,username,author_followers,text,created_at,retweets,replies,likes,quote_count,mentions_count,is_breaking,is_opinion,is_hashtag,hashtag_MeToo,hashtag_CriticsChoice,hashtag_CriticsChoiceAwards
0,FoxNews,20946694,FLASHBACK: Ex-intel officials claimed the Hunt...,2022-03-18 15:50:00+00:00,26,39,68,0,0,0,0,0,0,0,0
1,FoxNews,20946694,Biden must do more to counter Putin's propagan...,2022-03-18 15:45:00+00:00,23,94,92,7,0,0,0,0,0,0,0
2,FoxNews,20946694,NASCAR: What to expect from Atlanta's super ne...,2022-03-18 15:40:00+00:00,10,9,21,0,0,0,0,0,0,0,0
3,FoxNews,20946694,"BREAKING: Biden, Xi discuss Russia's war again...",2022-03-18 15:34:43+00:00,33,71,87,3,0,1,0,0,0,0,0
4,FoxNews,20946694,Transgender swimmer's NCAA championship win 'j...,2022-03-18 15:30:00+00:00,51,69,254,4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,nytimes,52297191,"Kyrylo Lysov, 22, and his girlfriend Yelyzavet...",2022-03-11 16:40:18+00:00,737,170,5277,54,0,0,0,0,0,0,0
1948,nytimes,52297191,Our journalists are looking into how companies...,2022-03-11 16:30:12+00:00,32,38,150,7,0,0,0,0,0,0,0
1949,nytimes,52297191,Breaking News: Alec Baldwin gave his most deta...,2022-03-11 16:18:19+00:00,59,81,227,20,0,1,0,0,0,0,0
1950,nytimes,52297191,A shipwreck from 1891 has been discovered in L...,2022-03-11 16:10:04+00:00,88,15,361,16,0,0,0,0,0,0,0


In [129]:
df.describe()

,author_followers,retweets,replies,likes,quote_count,mentions_count,is_breaking,is_opinion,is_hashtag,hashtag_MeToo,hashtag_CriticsChoice,hashtag_CriticsChoiceAwards
count,1.952000e+03,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000
mean,3.095251e+07,124.489242,132.122951,537.827357,21.152152,0.065061,0.014857,0.029201,0.004611,0.000512,0.004098,0.001537
std,1.461781e+07,356.144113,268.813812,1532.395478,54.889581,0.334836,0.121010,0.168412,0.067762,0.022634,0.063903,0.039183
min,2.094669e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.094669e+07,27.000000,30.000000,94.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.094670e+07,48.000000,58.000000,181.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.229719e+07,107.000000,121.000000,412.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.229719e+07,8422.000000,3851.000000,33729.000000,905.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
